In [9]:
import glob
import random
from vidaug import augmentors as va  # pip3 install git+https://github.com/okankop/vidaug --user
from PIL import Image
import hashlib
import os
import shutil
from moviepy.editor import ImageSequenceClip
from IPython.core.display import display, HTML


tmp_dir = 'tmp_media/'
shutil.rmtree(tmp_dir)
os.mkdir(tmp_dir)


class RGB2Gray(object):
    def __call__(self, clip):
        return [x.convert('L').convert('RGB') for x in clip]


def augmentation(prob=0.5, N=4, random_order=True):
    sometimes = lambda aug: va.Sometimes(prob, aug) # Used to apply augmentor with 50% probability
    return va.Sequential([
        va.SomeOf(
        [
            sometimes(va.GaussianBlur(sigma=3.0)),
            sometimes(va.ElasticTransformation(alpha=3.5, sigma=0.25)),
            sometimes(va.PiecewiseAffineTransform(displacement=5, displacement_kernel=1, displacement_magnification=1)),
            sometimes(va.RandomRotate(degrees=10)),
            sometimes(va.RandomResize(0.5)),
            sometimes(va.RandomTranslate(x=20, y=20)),
            sometimes(va.RandomShear(x=0.2, y=0.2)),
            # RandomCrop???
            sometimes(va.InvertColor()),
            sometimes(va.Add(100)),
            sometimes(va.Multiply(1.2)),
            sometimes(va.Pepper()),
            sometimes(va.Salt()),
            sometimes(va.HorizontalFlip()),
#             sometimes(va.Downsample(0.8)),
#             sometimes(va.Upsample(1.2)),
            sometimes(va.TemporalElasticTransformation()),
            sometimes(RGB2Gray())
        ],
        N=N,
        random_order=random_order
    )]) 


def save_images(images):
    image_paths = []
    for x in images:
        hashdigest = hashlib.md5(x.tobytes()).hexdigest()
        image_path = os.path.join(tmp_dir, f'{hashdigest}.jpg')
        x.save(image_path)
        image_paths.append(image_path)
    return image_paths


def load_batch(gif_id):
    image_paths = glob.glob(f'/home/ec2-user/mnt/giphy_dataset/{gif_id}/*.jpg')
    image_paths = sorted(image_paths, key=lambda x: int(x.split('/')[-1].split('.jpg')[0]))
    images = [Image.open(x) for x in image_paths]
    return images, save_images(images)


seq = augmentation()
videos = []
video_augs = []
for gif_id in ['2vsx3WK8jppXSDrb5t', '10002IKKbthZzG']:
    images, paths = load_batch(gif_id)
    if len(images) == 0:
        continue
    aug_images = seq(images)
    aug_paths = save_images(aug_images)
    
    original_clip = ImageSequenceClip(paths, fps=25)
    original_path = os.path.join(tmp_dir, f'{gif_id}_original.gif')
    original_clip.write_gif(original_path)
    
    aug_clip = ImageSequenceClip(aug_paths, fps=25)
    aug_path = os.path.join(tmp_dir, f'{gif_id}_aug.gif')
    aug_clip.write_gif(aug_path)
    display(HTML(f'<img src="tmp_media/{gif_id}_original.gif">'))
    display(HTML(f'<img src="tmp_media/{gif_id}_aug.gif">'))


[MoviePy] Building file tmp_media/2vsx3WK8jppXSDrb5t_original.gif with imageio


100%|██████████| 94/94 [00:10<00:00,  8.96it/s]



[MoviePy] Building file tmp_media/2vsx3WK8jppXSDrb5t_aug.gif with imageio


100%|██████████| 94/94 [00:06<00:00, 15.30it/s]



[MoviePy] Building file tmp_media/10002IKKbthZzG_original.gif with imageio


100%|██████████| 114/114 [00:06<00:00, 17.01it/s]



[MoviePy] Building file tmp_media/10002IKKbthZzG_aug.gif with imageio


100%|██████████| 114/114 [00:06<00:00, 18.65it/s]
